In [37]:
!pip3 install pytest-playwright

In [38]:
import os
from bs4 import BeautifulSoup
import requests
import time

In [39]:
letters = [chr(letter) for letter in range(ord('a'), ord('z')+1)]

In [41]:
def save_html(url, folder_path, file_name):
    # Make a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Create the folder if it doesn't exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # Write the HTML content to a file
        with open(os.path.join(folder_path, file_name), 'w', encoding='utf-8') as f:
            f.write(response.text)
    else:
        print("Failed to retrieve HTML. Status code:", response.status_code)

In [42]:
for letter in letters:
    url = f"https://www.basketball-reference.com/players/{letter}/"
    save_folder_path = "data/player_names"
    file_name = f"{letter}_names.html"

    save_html(url, save_folder_path, file_name)
    

In [43]:
def parse_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table with id="players"
    players_table = soup.find('table', id='players')
    if players_table:
        # Find all table rows within the players table
        table_rows = players_table.find_all('tr')
        
        # Extract href values from anchor elements within each row
        href_values = []
        for row in table_rows:
            anchor = row.find('a')
            if anchor and anchor.has_attr('href'):
                href_values.append(anchor['href'])
        
        return href_values
    else:
        print("No table with id='players' found in:", file_path)
        return []

In [44]:
for letter in letters:
    file_path = f"data/player_names/{letter}_names.html"
    href_values = parse_html_file(file_path)

    href_file_path = os.path.join("data/player_href", f"{letter}_names_href.txt")
    with open(href_file_path, 'w', encoding='utf-8') as href_file:
        href_file.write('\n'.join(href_values))